In [71]:
import pandas as pd
from functools import reduce
data=pd.read_csv("/Users/sy3115/Documents/Data/rAF_paper/finalized_project_data/gnomad.exomes.r2.1.1.sites_indelsonly_AC.txt",sep="\t",dtype={'CHR': 'str', 'POS': 'int','REF':'str','ALT':'str','AC':'int'})

/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/422564116.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv("/Users/sy3115/Documents/Data/rAF_paper/finalized_project_data/gnomad.exomes.r2.1.1.sites_indelsonly_AC.txt",sep="\t",dtype={'CHR': 'str', 'POS': 'int','REF':'str','ALT':'str','AC':'int'})


In [72]:
# This is how we add in the header 
headerList = ["CHR", "POS", "REF", "ALT", "AC"]
# Export a new csv file with the header 
data.to_csv("/Users/sy3115/Documents/Data/rAF_paper/finalized_project_data/2023-02-16_gnomad_indels_only_AC_header.csv", header = headerList, index = False)
# Read in the new csv file with the header into the same data variable 
data = pd.read_csv("/Users/sy3115/Documents/Data/rAF_paper/finalized_project_data/2023-02-16_gnomad_indels_only_AC_header.csv", sep = ",", dtype={'CHR': 'str', 'POS': 'int','REF':'str','ALT':'str','AC':'int'})
print(data.head())

  CHR    POS                            REF ALT  AC
0   1  12612                            GGT   G   0
1   1  12738                          CAGTG   C   1
2   1  12745  TGGGAGTGGCGTCGCCCCTAGGGCTCTAC   T   0
3   1  12758                              G  GC   1
4   1  12762                             CT   C   1


In [73]:
def cluster(pos_list, maxgap):
    pos=pos_list.astype(int)
    pos.sort_values(ascending=True, inplace=True)
    groups = [[list(pos)[0]]]
    for x in pos[1:]:
        if abs(x - groups[-1][-1]) <= maxgap:
            groups[-1].append(x)
        else:
            groups.append([x])
    return groups

def group_ids(pairs,bp):
    groups = cluster(pairs['POS'], bp)
    d = {}
    for i, group in enumerate(sorted(groups)):
        for elem in group:
            d[elem] = i
    return d


In [74]:
chrom=data['CHR'].unique()
bp5=5
bp10=10
bp15=15
bp20=20

chrom_output=[]
for i in chrom:
    print(i)
    input=data[data["CHR"]==i]
    
    ############# BP=5 ##########
    creategroups_bp5=group_ids(input,bp5)
    binned_groups_bp5 = pd.DataFrame(creategroups_bp5.items(), columns=['POS', 'GROUPID_bp5'])
    binned_groups_bp5['POS']=binned_groups_bp5['POS'].astype(int)
     
    ############# BP=10 ##########
    creategroups_bp10=group_ids(input,bp10)
    binned_groups_bp10 = pd.DataFrame(creategroups_bp10.items(), columns=['POS', 'GROUPID_bp10'])
    binned_groups_bp10['POS']=binned_groups_bp10['POS'].astype(int)
    
    ############# BP=15 ##########
    creategroups_bp15=group_ids(input,bp15)
    binned_groups_bp15 = pd.DataFrame(creategroups_bp15.items(), columns=['POS', 'GROUPID_bp15'])
    binned_groups_bp15['POS']=binned_groups_bp15['POS'].astype(int)
    
    ############# BP=20 ##########
    creategroups_bp20=group_ids(input,bp20)
    binned_groups_bp20 = pd.DataFrame(creategroups_bp20.items(), columns=['POS', 'GROUPID_bp20'])
    binned_groups_bp20['POS']=binned_groups_bp20['POS'].astype(int)
    
    #define list of DataFrames
    dfs = [input,binned_groups_bp5, binned_groups_bp10, binned_groups_bp15, binned_groups_bp20]

    #merge all DataFrames into one
    final_df = reduce(lambda  left,right: pd.merge(left,right,on=['POS'], how='outer'), dfs)
    final_df['AlleleCount_bp5']= final_df['AC'].groupby(final_df['GROUPID_bp5']).transform('sum')
    final_df['AlleleCount_bp10']= final_df['AC'].groupby(final_df['GROUPID_bp10']).transform('sum')
    final_df['AlleleCount_bp15']= final_df['AC'].groupby(final_df['GROUPID_bp15']).transform('sum')
    final_df['AlleleCount_bp20']= final_df['AC'].groupby(final_df['GROUPID_bp20']).transform('sum')
     
    chrom_output.append(final_df)
final_output = pd.concat(chrom_output)


1


/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for x in pos[1:]:
/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for x in pos[1:]:
/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, t

2


/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for x in pos[1:]:
/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for x in pos[1:]:
/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, t

3


...

/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for x in pos[1:]:
/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for x in pos[1:]:
/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, t

22
X


/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for x in pos[1:]:
/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for x in pos[1:]:
/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, t

Y


/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for x in pos[1:]:
/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for x in pos[1:]:
/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, t

In [75]:
final_output.to_csv("/Users/sy3115/Documents/Data/rAF_paper/finalized_project_data/gnomad.exomes.r2.1.1.sites_indelsonly_allele_counts.csv",index = False)

In [76]:
creategroups_bp10=group_ids(input,10)
binned_groups_bp10 = pd.DataFrame(creategroups_bp10.items(), columns=['POS', 'GROUPID_bp10'])

/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for x in pos[1:]:


In [77]:
binned_groups_bp10.head()

,POS,GROUPID_bp10
0,2709654,0
1,2709685,1
2,2710298,2
3,2710308,2
4,2710317,2


In [78]:
cluster(input['POS'].astype(int), bp10)

/var/folders/d3/vjc_50q165z6232gypbw3sq40000gp/T/ipykernel_96209/2148121841.py:5: FutureWarning: The behavior of `series[i:j]` with an integer-dtype index is deprecated. In a future version, this will be treated as *label-based* indexing, consistent with e.g. `series[i]` lookups. To retain the old behavior, use `series.iloc[i:j]`. To get the future behavior, use `series.loc[i:j]`.
  for x in pos[1:]:


[[2709654],
 [2709685],
 [2710298, 2710308, 2710308, 2710308, 2710317, 2710318],
 [2710330],
 [2713636, 2713641],
 [2713824],
 [2722817],
 [2733099],
 [2733323],
 [2734808, 2734818],
 [2829086],
 [2829108],
 [2829571],
 [2829675],
 [2843740],
 [2844681],
 [2844766],
 [2844901],
 [2845944],
 [2845975],
 [2846164],
 [3447518],
 [3447816],
 [4905037],
 [4905066],
 [4905346],
 [4905620, 4905620],
 [4924883],
 [4924913],
 [4925101],
 [4925116],
 [4925137],
 [4925320],
 [4925529, 4925538],
 [4966205],
 [4966635],
 [4966785],
 [4966877],
 [4967379],
 [4967666],
 [4967733],
 [4967948],
 [4968219, 4968223, 4968224],
 [4968253],
 [4968280, 4968281],
 [4968327, 4968328],
 [4968423],
 [4968758],
 [4968803],
 [4982523, 4982523, 4982523, 4982523, 4982523, 4982523],
 [5369214],
 [5449851],
 [5483262],
 [5491085],
 [5605435],
 [6114265],
 [6114280, 6114286],
 [6114520],
 [6114769],
 [6115358],
 [6115372],
 [6115490],
 [6115509],
 [6115620],
 [6736361, 6736370],
 [6736513],
 [6736775],
 [6737943],
 [67

In [79]:
[[input.iloc[0]]]

[[CHR          Y
  POS    2709654
  REF        CAG
  ALT          C
  AC           2
  Name: 1181648, dtype: object]]

In [80]:
input['POS'].astype(int)

1181648     2709654
1181649     2709685
1181650     2710298
1181651     2710308
1181652     2710308
             ...   
1182136    26942287
1182137    26951069
1182138    27010566
1182139    27010566
1182140    27190084
Name: POS, Length: 493, dtype: int64

In [81]:
pos=input['POS'].astype(int)
print(pos)
pos.sort_values(ascending=True, inplace=True)
print(pos)

1181648     2709654
1181649     2709685
1181650     2710298
1181651     2710308
1181652     2710308
             ...   
1182136    26942287
1182137    26951069
1182138    27010566
1182139    27010566
1182140    27190084
Name: POS, Length: 493, dtype: int64
1181648     2709654
1181649     2709685
1181650     2710298
1181651     2710308
1181652     2710308
             ...   
1182136    26942287
1182137    26951069
1182138    27010566
1182139    27010566
1182140    27190084
Name: POS, Length: 493, dtype: int64


In [82]:
final_output.head(200)

,CHR,POS,REF,ALT,AC,GROUPID_bp5,GROUPID_bp10,GROUPID_bp15,GROUPID_bp20,AlleleCount_bp5,AlleleCount_bp10,AlleleCount_bp15,AlleleCount_bp20
0,1,12612,GGT,G,0,0,0,0,0,0,0,0,0
1,1,12738,CAGTG,C,1,1,1,1,1,1,1,3,3
2,1,12745,TGGGAGTGGCGTCGCCCCTAGGGCTCTAC,T,0,2,1,1,1,0,1,3,3
3,1,12758,G,GC,1,3,2,1,1,2,2,3,3
4,1,12762,CT,C,1,3,2,1,1,2,2,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,1,877480,C,CG,4,126,114,99,91,20,21,21,21
196,1,877480,CG,C,11,126,114,99,91,20,21,21,21
197,1,877480,CGG,C,1,126,114,99,91,20,21,21,21
198,1,877487,GA,G,1,127,114,99,91,1,21,21,21


In [83]:
binned_groups_bp15.head()

,POS,GROUPID_bp15
0,2709654,0
1,2709685,1
2,2710298,2
3,2710308,2
4,2710317,2


In [84]:
final_output.head(100)

,CHR,POS,REF,ALT,AC,GROUPID_bp5,GROUPID_bp10,GROUPID_bp15,GROUPID_bp20,AlleleCount_bp5,AlleleCount_bp10,AlleleCount_bp15,AlleleCount_bp20
0,1,12612,GGT,G,0,0,0,0,0,0,0,0,0
1,1,12738,CAGTG,C,1,1,1,1,1,1,1,3,3
2,1,12745,TGGGAGTGGCGTCGCCCCTAGGGCTCTAC,T,0,2,1,1,1,0,1,3,3
3,1,12758,G,GC,1,3,2,1,1,2,2,3,3
4,1,12762,CT,C,1,3,2,1,1,2,2,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,721390,C,CA,1,71,66,55,49,1,1,1,1
96,1,721573,CA,C,1,72,67,56,50,1,1,1,1
97,1,721652,AT,A,1,73,68,57,51,1,1,1,1
98,1,721724,A,AT,363,74,69,58,52,363,363,363,363
